In [ ]:
!pip install ultralytics
!pip install gradio
!pip install pandas
!pip install deepface
!pip install pillow
!pip install opencv-python
!pip install face_recognition

In [ ]:
import os
import cv2
import shutil
import face_recognition

def detect_faces_face_recognition(image_path):
    """Detect faces in an image using the face_recognition library and return the face encodings."""
    image = face_recognition.load_image_file(image_path)
    face_locations = face_recognition.face_locations(image)
    face_encodings = face_recognition.face_encodings(image, face_locations)
    return face_encodings

def copy_matching_images(input_image_path, album_dir, output_dir, tolerance=0.5):
    """Copy matching images from album directory to output directory based on face recognition."""
    os.makedirs(output_dir, exist_ok=True)

    print("Finding matches for:", input_image_path)

    # Detect and encode the face(s) in the input image
    input_encodings = detect_faces_face_recognition(input_image_path)

    if not input_encodings:
        print("No faces detected in the input image.")
        return

    input_encoding = input_encodings[0]  # Assuming we are using the first detected face

    # Iterate through all images in the album directory
    for filename in os.listdir(album_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            album_image_path = os.path.join(album_dir, filename)

            # Detect faces and get encodings for each face in the album image
            album_encodings = detect_faces_face_recognition(album_image_path)

            # Check if any face in the album matches the input face
            match_found = any(face_recognition.compare_faces([input_encoding], album_encoding, tolerance=tolerance)[0]
                              for album_encoding in album_encodings)

            if match_found:
                print(f"Match found: {filename}")
                shutil.copy(album_image_path, os.path.join(output_dir, filename))
            else:
                print(f"No match for: {filename}")

    print("Matching process completed.")

if __name__ == "__main__":
    input_image_path = input("Enter the path to the input image: ").strip()
    album_dir = input("Enter the directory of the photo shoot album: ").strip()
    output_dir = input("Enter the output directory for matching images: ").strip()

    if not os.path.isfile(input_image_path):
        print("The specified input image does not exist.")
    else:
        # Run the matching process
        copy_matching_images(input_image_path, album_dir, output_dir)

    # Print the results (list of copied images)
    print("\nMatching Images Copied to", output_dir + ":")
    for filename in os.listdir(output_dir):
        print(filename)